# Stochastic Variational Inference for LDA

## Introducción

En el presente notebook se realiza una implementación del algoritmo para topic modelling, Latent Dirichlet Allocation (LDA), en su versión con Stochastic Variational Inference. La implementación trata de apegarse a lo propuesto por Hoffman, Blei y Wang en [Stochastic Variational Inference](http://www.columbia.edu/~jwp2128/Papers/HoffmanBleiWangPaisley2013.pdf).
Otros trabajos que fueron tomados como referencia son:
 * Para un panorama más amplio sobre LDA: [Inference Methods for Latent Dirichlet Allocation](http://times.cs.uiuc.edu/course/598f16/notes/lda-survey.pdf);
 * Para una explicación más detallada sobre la Mean-Field Variational Family: [Variational Inference: A Review for Statisticians](https://arxiv.org/pdf/1601.00670.pdf)
 
Adicionalmente, la idea de implementar LDA surgió de un intento previo de querer implementar la propuesta de Wang y Blei, en [Collaborative Topic Modeling for Recommending Scientific Articles](http://www.cs.columbia.edu/~blei/papers/WangBlei2011.pdf), para Sistemas de Recomendación. De hecho el dataset utilizado aqui es el mismo que usan ellos. No descarto implementar este paper como siguiente paso.

## Marco teórico

Asumiendo K tópicos, y D documentos, cada uno con N palabras (por simplicidad asumo documentos de igual longitud) pertenecientes a un vocabulario de tamaño V; el modelo generativo es el siguiente:

1. Generamos tópicos $\beta_k$ ~ Dirichlet($\eta,\dots,\eta$) para cada $k\in \{0,\dots,K-1\}$
2. Para cada documento $d \in \{0,\dots, D-1\}$:
    3. Generamos $\theta_d$ ~ Dirichlet($\alpha,\dots,\alpha$)
    4. Para cada palabra $w \in \{0,\dots, N-1\}$:
        5. Generamos la asignación de tema $z_{dn}$ ~ Multinomial($\theta_d$)
        6. Generamos la palabra $w_{dn}$ ~ Multinomial($\beta_{z_{dn}}$)

Los parámetros involucrados son:
* $\beta_k$, el tópico $k$. Un tópico consiste básicamente en un vector de probabilidad de longitud V, que modela la distribución de las distintas palabras en el vocabulario para ese tema
* $\theta_d$, la proporción de los tópicos para el documento $d$. Nuevamente es un vector de probabilidades, que en este caso apunta a modelar la participación parcial de los distintos temas para un mismo documento. El hecho de que se permita que un documento pertenezca a un mixture de tópicos en lugar de uno solo, es una de las principales virtudes de LDA. __Esta es la principal variable sobre la que nos interesa poder hacer inferencia__.
* $z_{dn}$, es la asignación del tópico de la palabra $n$ en el domunento $d$
* $w_{dn}$, es la palabra $n$ del documento $d$. Pertenece al tópico $z_{dn}$

## Dataset

El dataset fue descargado de http://www.cs.cmu.edu/~chongw/data/citeulike/. El mismo contiene información del sitio http://www.citeulike.org/, que le permite a investigadores armar sus bibliotecas de papers y recibir recomendaciones en base a las mismas. En particular yo no uso la información referente a los usuarios y sus bibliotecas, si no que únicamente me centro en la data sobre artículos.
Los dos archivos que uso son:
* mult.dat, que contiene los ids de las palabras más relevantes de cada artículos, y sus conteos. En total son 16980 artículos (documentos).
* vocab.dat, el mapeo de los ids a la palabra específica. El tamaño del vocabulario usado es de 8000 palabras.

La extracción del vocabulario, y demás preprocesamientos de la data son los que se explican en la sección 4 de [Wang and Blei (2011)](http://www.cs.columbia.edu/~blei/papers/WangBlei2011.pdf).

Adicionalmente, en esta primera iteración del algoritmo, pido que todos los documentos tengan la misma cantidad de palabras. Eso me lleva a realizar un procesamiento extra, en el cual descarto documentos con menos de 40 palabras, quedandome con 14836. A su vez, para estos artículos, solo conservo las primeras 40 palabras, ordenadas de mayor a menor ocurrencia en el documento.

A continuación, importo algunas bibliotecas que voy a usar, y realizo la carga del dataset. Por cuestiones de reproducibilidad, seteo una semilla fija.

In [1]:
from scipy.stats import dirichlet, multinomial, expon
from scipy.special import digamma
from collections import defaultdict
from math import pow
import numpy as np
import copy
import logging

from online_lda import OnlineLDA
from mini_batch_lda import MiniBatchLDA
from helpers import *

logging.basicConfig(level=logging.INFO)

In [2]:
def load_dataset():
    words_per_doc = defaultdict(list)
    # Parse line by line the documents data's file
    with open('data/mult.dat') as keywords_f:
        words_and_counts_by_doc = []
        for l in keywords_f:
            data = l.split() # First element is the number of distinct words, the rest are the word_ids with its counters 
            words_and_counts_by_doc.append(data[1:])
        for d, words_and_counts in enumerate(words_and_counts_by_doc):
            for w_c in words_and_counts:
                word, cnt = w_c.split(':')
                for _ in xrange(int(cnt)):    
                    words_per_doc[d].append(int(word))
            # Resampling the data to always have the same number of words per document
            np.random.seed(42)
            words_per_doc[d] = np.random.choice(words_per_doc[d], size=60)
    
    vocab = {}
    with open('data/vocab.dat') as vocab_f:
        for v, word in enumerate(vocab_f):
            vocab[v] = word.rstrip()
    
    return words_per_doc, vocab
            

In [3]:
words_by_doc, vocab = load_dataset()

## LDA

Primero me defino algunas funciones auxiliares que me van a servir luego.

La clase _LatentDirichletAllocation_ tiene como método principal _fit_, que es el que se encarga de aplicar el algoritmo propuesto por Hoffman et al. (2013). La notación elegida para los parámetros sigue fielmente la usada en la sección 3 de dicho paper. 

La imagen siguiente muestra el pseudocódigo del algoritmo en cuestión.

![title](img/SVI-pseudocode.png)

In [4]:
D = len(words_by_doc)  # number of documents
N = 60 # words per doc
K = 60  # number of topics
V = len(vocab)  # vocabulary size

# For alpha and eta values I follow recommendation from Griffiths and Steyvers (2004)
lda = OnlineLDA(K, D, N, V, 50.0/K, 0.1, words_by_doc)
lda.fit()

Iteracion: 0
Documento: 5770
Ro: 0.0334929207043
Iteracion: 1
Documento: 11239
Ro: 0.0333109641294
Iteracion: 2
Documento: 11493
Ro: 0.0331313897449
Iteracion: 3
Documento: 661
Ro: 0.0329541483957
Iteracion: 4
Documento: 12864
Ro: 0.032779192306
Iteracion: 5
Documento: 7180
Ro: 0.0326064750308
Iteracion: 6
Documento: 363
Ro: 0.0324359514088
Iteracion: 7
Documento: 4779
Ro: 0.0322675775179
Iteracion: 8
Documento: 5343
Ro: 0.0321013106322
Iteracion: 9
Documento: 1663
Ro: 0.0319371091812
Iteracion: 10
Documento: 6505
Ro: 0.03177493271
Iteracion: 11
Documento: 14368
Ro: 0.0316147418418
Iteracion: 12
Documento: 9360
Ro: 0.0314564982415
Iteracion: 13
Documento: 13164
Ro: 0.0313001645812
Iteracion: 14
Documento: 11476
Ro: 0.0311457045065
Iteracion: 15
Documento: 7843
Ro: 0.0309930826047
Iteracion: 16
Documento: 4408
Ro: 0.0308422643739
Iteracion: 17
Documento: 11600
Ro: 0.0306932161937
Iteracion: 18
Documento: 4302
Ro: 0.0305459052965
Iteracion: 19
Documento: 6823
Ro: 0.0304002997402
Iteracio

KeyboardInterrupt: 

In [ ]:
for i, lamb_k in enumerate(lda.lamb):
    print_md('Topic {}'.format(i), bold=True)
    np.random.seed(42)
    words_distribution = np.random.dirichlet(lamb_k)
    #print words_distribution.argsort()[::-1][:10]
    #words_distribution.sort()
    #print words_distribution[::-1][:10]
    print_words(words_distribution.argsort()[::-1][:20], vocab)

In [5]:
D = len(words_by_doc)  # number of documents
N = 60 # words per doc
K = 60  # number of topics
V = len(vocab)  # vocabulary size
minibatch_size = 64

# For alpha and eta values I follow suggestion from Griffiths and Steyvers (2004)
lda = MiniBatchLDA(K, D, N, V, 50.0/K, 0.1, words_by_doc, minibatch_size)
lda.fit()

INFO:mini_batch_lda:Iteration: 0
INFO:mini_batch_lda:Ro: 1.0
INFO:mini_batch_lda:Iteration: 1
INFO:mini_batch_lda:Ro: 0.574349177499
INFO:mini_batch_lda:Iteration: 2
INFO:mini_batch_lda:Ro: 0.415243646539
INFO:mini_batch_lda:Iteration: 3
INFO:mini_batch_lda:Ro: 0.329876977693
INFO:mini_batch_lda:Iteration: 4
INFO:mini_batch_lda:Ro: 0.275945932292
INFO:mini_batch_lda:Iteration: 5
INFO:mini_batch_lda:Ro: 0.238494846851
INFO:mini_batch_lda:Iteration: 6
INFO:mini_batch_lda:Ro: 0.210824737371
INFO:mini_batch_lda:Iteration: 7
INFO:mini_batch_lda:Ro: 0.189464570814
INFO:mini_batch_lda:Iteration: 8
INFO:mini_batch_lda:Ro: 0.172427285991
INFO:mini_batch_lda:Iteration: 9
INFO:mini_batch_lda:Ro: 0.158489319246
INFO:mini_batch_lda:Iteration: 10
INFO:mini_batch_lda:Ro: 0.1468540242
INFO:mini_batch_lda:Iteration: 11
INFO:mini_batch_lda:Ro: 0.136979319126
INFO:mini_batch_lda:Iteration: 12
INFO:mini_batch_lda:Ro: 0.128482896333
INFO:mini_batch_lda:Iteration: 13
INFO:mini_batch_lda:Ro: 0.121087014505
I

In [6]:
for i, lamb_k in enumerate(lda.lamb):
    print_md('Topic {}'.format(i), bold=True)
    np.random.seed(42)
    words_distribution = np.random.dirichlet(lamb_k)
    #print words_distribution.argsort()[::-1][:10]
    #words_distribution.sort()
    #print words_distribution[::-1][:10]
    print_words(words_distribution.argsort()[::-1][:20], vocab)

**Topic 0**

52, development
382, linear
471, project
286, recognition
641, tested
448, input
537, noise
694, experiment
506, coding
624, rapid
867, file
571, hypothesis
709, testing
659, representations
756, climate
666, stochastic
1221, creation
1166, speed
1229, coupled
1784, allowed


**Topic 1**

99, transcription
163, mechanisms
185, distribution
256, targets
122, mirnas
144, mirna
1026, exist
405, mapping
585, degree
461, approximately
741, micrornas
970, reveals
747, correlated
595, mouse
817, promoter
71, cells
417, expressed
1054, units
954, microrna
1183, understood


**Topic 2**

...

86, recent
49, complex
77, features
170, years
373, images
339, shown
300, interest
379, attention
323, organization
480, central
667, theories
702, tests
957, package
894, mean
821, interactive
849, ones
1020, face
454, average
1010, capture
1213, reconstruction


**Topic 58**

7, results
258, presented
219, scientific
363, machine
378, area
644, evaluate
492, discussed
411, events
409, public
590, implemented
759, flow
988, capacity
983, wikipedia
46, provide
906, efforts
1023, benefits
1125, index
707, working
1031, age
1167, diffusion


**Topic 59**

129, individual
132, terms
186, order
210, image
248, relationships
486, map
651, guide
420, feature
895, matching
863, idea
796, surface
695, term
668, shared
866, medical
780, interpretation
1126, supports
1665, running
1145, contribution
1410, navigation
1128, modelling


In [20]:
lda.gamma[90:100]

array([[ 1.80063452,  0.96543904,  4.6927096 ,  0.83347637,  0.8334179 ,
         0.84745332,  0.86911628,  2.74826727,  0.869611  ,  0.84137215,
         3.76127965,  0.85443808,  2.92360381,  0.83878002,  0.87534081,
         1.74202782,  0.83826289,  0.84145542,  0.83606442,  0.84334155,
         0.83470439,  0.83623865,  0.83370108,  1.8481884 ,  3.32069561,
         0.83683384,  6.56288889,  2.76104599,  1.87336555,  0.83972944,
         0.83646284,  2.90549102,  0.92129091,  0.84222479,  1.64027932,
         0.84410728,  0.8525896 ,  0.83425007,  0.86161461,  1.86977874,
         1.84935466,  0.83918701,  7.56134089,  0.84088599,  3.95019911,
         0.84304385,  1.30005764,  0.83810353,  0.85924589,  0.83629584,
         3.0258385 ,  2.89843033,  2.58121708,  0.83400247,  0.83747627,
         5.88358516,  2.81906587,  4.11606799,  2.84057258,  1.83445639],
       [ 2.83778888,  0.85705955,  0.84069184,  2.6649871 ,  1.85947524,
         0.83679065,  2.82190297,  5.82438299,  2.